In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
spark = SparkSession.builder.appName("loa") \
.master("spark://10.208.36.84:7077") \
.config("spark.hadoop.defaultFS","hdfs://10.208.36.84:9000") \
.config("spark.executor.cores",2) \
.config("spark.executor.memory","4g") \
.config("spark.cores.max","4") \
.getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
sche = "member_id string, delinq_2yrs float,delinq_amnt float,pub_rec float,pub_rec_bankruptcies float,inq_last_6mths float,total_rec_late_fee float,mths_since_last_delinq float,mths_since_last_record float"      

In [4]:
df = spark.read.format("csv").option("header","true") \
.schema(sche)\
.load("/home/tushar/Documents/project/loan_deafulters_raw")

In [5]:
df.printSchema()

root
 |-- member_id: string (nullable = true)
 |-- delinq_2yrs: float (nullable = true)
 |-- delinq_amnt: float (nullable = true)
 |-- pub_rec: float (nullable = true)
 |-- pub_rec_bankruptcies: float (nullable = true)
 |-- inq_last_6mths: float (nullable = true)
 |-- total_rec_late_fee: float (nullable = true)
 |-- mths_since_last_delinq: float (nullable = true)
 |-- mths_since_last_record: float (nullable = true)



In [6]:
df.count()

868845

In [7]:
df = df.withColumn("delinq_2yrs",col("delinq_2yrs").cast("int")).fillna(0,subset = ["delinq_2yrs"])

In [8]:
df.createOrReplaceTempView("def")

In [9]:
default_enquiry= spark.sql("select member_id from def where pub_rec>0 or pub_rec_bankruptcies>0 or inq_last_6mths>0")

In [10]:
default_enquiry.count()

395669

In [11]:
defaulters= spark.sql("select member_id,delinq_2yrs,delinq_amnt,int(mths_since_last_delinq) from def where delinq_2yrs>0 or delinq_amnt>0 or mths_since_last_delinq>0") 

In [12]:
defaulters.printSchema()

root
 |-- member_id: string (nullable = true)
 |-- delinq_2yrs: integer (nullable = true)
 |-- delinq_amnt: float (nullable = true)
 |-- mths_since_last_delinq: integer (nullable = true)



In [13]:
defaulters.write.format("parquet").mode("overwrite").option("path","/home/tushar/Documents/project/cleaned_defaulters/parquet").save()

In [14]:
defaulters.write.format("csv").mode("overwrite").option("path","/home/tushar/Documents/project/cleaned_defaulters/csv").save()

In [15]:
default_enquiry.write.format("parquet").mode("overwrite").option("path","/home/tushar/Documents/project/cleaned_enquiry_defaulters/parquet").save()

In [16]:
default_enquiry.write.format("csv").mode("overwrite").option("path","/home/tushar/Documents/project/cleaned_enquiry_defaulters/csv").save()

In [17]:
df = df.withColumn("pub_rec",col("pub_rec").cast("int")).fillna(0,subset = ["pub_rec"])

In [18]:
df = df.withColumn("pub_rec_bankruptcies",col("pub_rec_bankruptcies").cast("int")).fillna(0,subset = ["pub_rec_bankruptcies"])

In [19]:
df = df.withColumn("inq_last_6mths",col("inq_last_6mths").cast("int")).fillna(0,subset = ["inq_last_6mths"])

In [20]:
df.createOrReplaceTempView("loan_def")

In [21]:
default_enquiry_detail = spark.sql("select member_id,pub_rec,pub_rec_bankruptcies,inq_last_6mths from loan_def ")

In [22]:
default_enquiry_detail.write.format("parquet").mode("overwrite").option("path","/home/tushar/Documents/project/cleaned_def_enq_detail/parquet").save()

In [ ]:
default_enquiry_detail.write.format("csv").mode("overwrite").option("path","/home/tushar/Documents/project/cleaned_def_enq_detail/csv").save()